In [1]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed


In [96]:
dataset_fileset = glob(os.path.join("dataset", "prediction_result", "test", "*.npy"))
dataset_fileset.sort(key=lambda x : int(x.split("\\")[-1].split("_")[0]))
get_size = np.load(dataset_fileset[0])
data = np.empty((get_size.shape[0]*len(dataset_fileset), get_size.shape[1]))
for idx, dataset_file in enumerate(dataset_fileset):
    print(f'now processing {dataset_file}')
    data[idx*get_size.shape[0]:(idx+1)*get_size.shape[0]] = np.load(dataset_file)


now processing dataset\prediction_result\test\0_blc_138.npy
now processing dataset\prediction_result\test\0_blc_139.npy
now processing dataset\prediction_result\test\0_blc_141.npy
now processing dataset\prediction_result\test\0_blc_143.npy
now processing dataset\prediction_result\test\0_blc_145.npy
now processing dataset\prediction_result\test\0_blc_147.npy
now processing dataset\prediction_result\test\0_blc_149.npy
now processing dataset\prediction_result\test\0_blc_151.npy
now processing dataset\prediction_result\test\0_blc_153.npy
now processing dataset\prediction_result\test\0_blc_155.npy
now processing dataset\prediction_result\test\0_blc_156.npy
now processing dataset\prediction_result\test\0_blc_158.npy
now processing dataset\prediction_result\test\0_blc_160.npy
now processing dataset\prediction_result\test\0_blc_162.npy
now processing dataset\prediction_result\test\0_blc_164.npy
now processing dataset\prediction_result\test\0_blc_166.npy
now processing dataset\prediction_result

In [72]:
def get_similarity(idx1, spec1, spec1_SO2):
    topK = 10
    compare_data = data[np.where(data[:,41]!=spec1_SO2)]
    diff_spec = (spec1[:40] - compare_data[:, :40])/np.abs(compare_data[:, :40])*100
    similarity = np.mean(np.abs(diff_spec), axis=1)
    similarity_total_idx = np.argsort(similarity)
    similarity_topK_idx = similarity_total_idx[:topK]

    similarity_topK = similarity[similarity_topK_idx]
    spec_topK = compare_data[similarity_topK_idx]
    similarity_mean = np.mean(similarity)
    similarity_std = np.std(similarity)
    row = {'similarity_mean' : similarity_mean, 'similarity_std': similarity_std}
    for idx in range(spec1.shape[0]):
        row[f'spec1_value_{idx}'] = spec1[idx]
    for idx in range(similarity_topK.shape[0]):
        row[f'similarity_{idx}'] = similarity_topK[idx]
    for i in range(spec_topK.shape[0]):
        for j in range(spec_topK.shape[1]):
            row[f'similar_spec{i}_value_{j}'] = spec_topK[i,j]
    
    return row


data = data[~np.any(data[:,:40] == 0, axis=1)] # remove any zero reflectance
res = Parallel(n_jobs=-5)(delayed(get_similarity)(idx1, data[idx1], data[idx1,41]) for idx1 in tqdm(range(data[:100].shape[0])))
res = pd.DataFrame(res)
subject = 'ctchen'
result = 'prediction_model_formula2'
os.makedirs(os.path.join('result', subject, result), exist_ok=True)
res.to_csv(os.path.join('result', subject, result), index=False)


100%|██████████| 100/100 [00:14<00:00,  7.06it/s]


In [86]:
r[]

(0,
 similarity_mean            505.173940
 similarity_std            5146.556204
 spec1_value_0                0.886804
 spec1_value_1                0.839706
 spec1_value_2                0.715375
                              ...     
 similar_spec9_value_38      -0.106929
 similar_spec9_value_39      -0.137685
 similar_spec9_value_40     139.000000
 similar_spec9_value_41      -0.270000
 similar_spec9_value_42     141.000000
 Name: 0, Length: 485, dtype: float64)

In [89]:
np.round(100*data[:,41])

array([-30., -29., -28., ...,  18.,  19.,  20.])

In [80]:
res = pd.DataFrame(res)
test = {'test' : [i for i in range(100)]}
test = pd.DataFrame(test)
pd.concat((res,test),axis=1)

,similarity_mean,similarity_std,spec1_value_0,spec1_value_1,spec1_value_2,spec1_value_3,spec1_value_4,spec1_value_5,spec1_value_6,spec1_value_7,...,similar_spec9_value_34,similar_spec9_value_35,similar_spec9_value_36,similar_spec9_value_37,similar_spec9_value_38,similar_spec9_value_39,similar_spec9_value_40,similar_spec9_value_41,similar_spec9_value_42,test
0,505.173940,5146.556204,0.886804,0.839706,0.715375,0.542545,0.452722,0.376717,0.397799,0.449445,...,0.043909,-0.017672,-0.085598,-0.118482,-0.106929,-0.137685,139.0,-0.27,141.0,0
1,490.976677,5021.546405,0.873398,0.829422,0.691366,0.524347,0.438572,0.364163,0.384551,0.434314,...,0.044860,-0.018265,-0.074202,-0.117401,-0.104728,-0.122752,138.0,-0.26,73.0,1
2,476.458245,4876.124081,0.859704,0.808333,0.667381,0.506201,0.424251,0.351567,0.371295,0.419448,...,0.036128,-0.018343,-0.074748,-0.109845,-0.097210,-0.123734,139.0,-0.26,73.0,2
3,458.498312,4534.089666,0.846302,0.779194,0.643462,0.488061,0.410005,0.338968,0.357843,0.404283,...,0.039050,-0.015404,-0.076150,-0.102513,-0.095034,-0.122416,139.0,-0.24,141.0,3
4,446.311320,4563.334397,0.832609,0.749804,0.619548,0.469934,0.396075,0.326317,0.344648,0.389214,...,0.039331,-0.018498,-0.075513,-0.110864,-0.094340,-0.121441,138.0,-0.24,141.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,386.392518,4466.588647,-0.866277,-0.746904,-0.693834,-0.597711,-0.507431,-0.396836,-0.349407,-0.266160,...,-0.043085,0.010400,0.040169,0.034552,0.054910,0.071163,138.0,0.15,73.0,95
96,412.097448,4773.250634,-0.956386,-0.810843,-0.746441,-0.637795,-0.538898,-0.425053,-0.379174,-0.299943,...,-0.053147,0.012212,0.040732,0.040413,0.063700,0.083515,138.0,0.16,15.0,96
97,438.163858,5080.173637,-1.046791,-0.874344,-0.799187,-0.677919,-0.570504,-0.453360,-0.409082,-0.333661,...,-0.053261,0.007700,0.039683,0.042567,0.066995,0.084620,139.0,0.19,0.0,97
98,465.134579,5393.021940,-1.136815,-0.938163,-0.851768,-0.717897,-0.602201,-0.481506,-0.438815,-0.367254,...,-0.047716,0.006393,0.030867,0.042986,0.067425,0.086912,139.0,0.17,141.0,98
